In [26]:
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores.faiss import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [27]:
loader = TextLoader("data/speech.txt")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = text_splitter.split_documents(documents)

In [28]:
embedding = OllamaEmbeddings(model="llama3")
db = FAISS.from_documents(documents, embedding)

In [29]:
query = "What does the speaker believe is the main reason the United States should enter the war?"
query_result = db.similarity_search(query, k=1)

In [30]:
print(query_result[0].page_content)

The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.


In [35]:
retriever = db.as_retriever()
query_result = retriever.invoke(query)

In [36]:
documents_and_scores = db.similarity_search_with_score(query)
documents_and_scores

[(Document(id='088fa34f-f0ff-4a0d-bb35-ff62f998b241', metadata={'source': 'data/speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
  np.float32(20637.043)),
 (Document(id='63768088-ffb2-4256-8d44-f5c4979a70cd', metadata={'source': 'data/speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and t

In [42]:
document = documents_and_scores[0][0]
score = documents_and_scores[0][1]
document, score

(Document(id='088fa34f-f0ff-4a0d-bb35-ff62f998b241', metadata={'source': 'data/speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
 np.float32(20637.043))

In [45]:
embedded_query = embedding.embed_query(query)
query_result = db.similarity_search_by_vector(embedded_query)
query_result

[Document(id='088fa34f-f0ff-4a0d-bb35-ff62f998b241', metadata={'source': 'data/speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
 Document(id='63768088-ffb2-4256-8d44-f5c4979a70cd', metadata={'source': 'data/speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treas

In [46]:
db.save_local("data/faiss_index")

In [50]:
db = FAISS.load_local("data/faiss_index", embeddings=embedding, allow_dangerous_deserialization=True)

In [51]:
query_result = db.similarity_search(query)
query_result

[Document(id='088fa34f-f0ff-4a0d-bb35-ff62f998b241', metadata={'source': 'data/speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
 Document(id='63768088-ffb2-4256-8d44-f5c4979a70cd', metadata={'source': 'data/speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treas